In [65]:
import pandas as pd
import numpy as np
from utils import peek
import config
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV, learning_curve, LearningCurveDisplay, StratifiedKFold
import pickle
import optuna
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgbm
from catboost import CatBoostClassifier, Pool
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore", module="optuna")
warnings.filterwarnings("ignore", module="lightgbm")

## Baseline

In [76]:
train = pd.read_csv(config.TRAIN)

In [17]:
def train_lr_cv(X, y, param, verbose=True):
    clf = LogisticRegression(**param)
    cvs = []
    for fold in range(0, 5):
        train_x = X.query(f'fold != {fold}').drop(columns=['customer_phone', 'fold'])
        train_y = y[train_x.index]
        valid_x = X.query(f'fold == {fold}').drop(columns=['customer_phone', 'fold'])
        valid_y = y[valid_x.index]
        valid_score = clf.fit(train_x, train_y).predict_proba(valid_x)[:, 1]
        cvs.append(roc_auc_score(valid_y, valid_score))
    
    if verbose:
        print("five fold auc:", *cvs, sep=", ")
        print("mean auc:", np.mean(cvs))
        
    return cvs

In [18]:
def train_lr(X, y, param):
    clf = LogisticRegression(**param)
    clf.fit(X, y)
    return clf

In [77]:
param = {'C':1, 'max_iter':20, 'solver':'newton-cg', 'class_weight':'balanced'}
cvs = train_lr_cv(train.drop('order_status_key', axis=1), train['order_status_key'], param)

five fold auc:, 0.7721792687721137, 0.7345826235093696, 0.7768851430108491, 0.7359722189073725, 0.7388064154275759
mean auc: 0.7516851339254561


In [78]:
param = {'C':1, 'max_iter':20, 'solver':'newton-cg', 'class_weight':'balanced'}
model = train_lr(train.drop(['order_status_key', 'fold', 'customer_phone'], axis=1), train['order_status_key'], param)

In [79]:
pickle.dump(model, open(config.BASELINE, "wb"))

## Random Forest

In [83]:
train = pd.read_csv(config.TRAIN2)

In [84]:
# tune using optuna
clf = RandomForestClassifier(random_state=42, class_weight="balanced", criterion='entropy', n_jobs=-1)
param_distributions = {
    'n_estimators':optuna.distributions.IntDistribution(100, 500, log=True),
    'max_depth':optuna.distributions.IntDistribution(5, 10, step=1), 
    'min_samples_split':optuna.distributions.IntDistribution(5, 50, step=5), 
    'min_samples_leaf':optuna.distributions.IntDistribution(1, 10, step=1),
    'max_features':optuna.distributions.FloatDistribution(0.2, 0.6, step=0.1),
    'max_samples':optuna.distributions.FloatDistribution(0.4, 0.7, step=0.1)
}
optuna_search = optuna.integration.OptunaSearchCV(clf, param_distributions, n_trials=100, scoring="roc_auc", n_jobs=-1)
X, y = train.drop(['order_status_key', 'fold', 'customer_phone'], axis=1), train['order_status_key']
optuna_search.fit(X, y)
y_pred = optuna_search.predict(X)

/tmp/ipykernel_207703/3430836499.py:11: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = optuna.integration.OptunaSearchCV(clf, param_distributions, n_trials=100, scoring="roc_auc", n_jobs=-1)
[I 2023-05-25 17:43:48,240] A new study created in memory with name: no-name-9963dbad-bafd-4cb4-969e-a03e2166e0e0
[I 2023-05-25 17:44:35,349] Trial 13 finished with value: 0.7490954593770807 and parameters: {'n_estimators': 112, 'max_depth': 6, 'min_samples_split': 30, 'min_samples_leaf': 4, 'max_features': 0.6, 'max_samples': 0.7}. Best is trial 13 with value: 0.7490954593770807.
[I 2023-05-25 17:44:36,181] Trial 20 finished with value: 0.7482814265029307 and parameters: {'n_estimators': 118, 'max_depth': 5, 'min_samples_split': 20, 'min_samples_leaf': 3, 'max_features': 0.6, 'max_samples': 0.7}. Best is trial 13 with value: 0.7490954593770807.
[I 2023-05-25 17:44:37,039] Trial 27 finished with value: 0.751658

In [85]:
optuna_search.best_params_

{'n_estimators': 233,
 'max_depth': 10,
 'min_samples_split': 45,
 'min_samples_leaf': 2,
 'max_features': 0.2,
 'max_samples': 0.4}

In [86]:
def train_rf_cv(X, y, param, verbose=True):
    clf = RandomForestClassifier(**param)
    cvs = []
    for fold in range(0, 5):
        train_x = X.query(f'fold != {fold}').drop(columns='fold')
        train_y = y[train_x.index]
        valid_x = X.query(f'fold == {fold}').drop(columns='fold')
        valid_y = y[valid_x.index]
        valid_score = clf.fit(train_x, train_y).predict_proba(valid_x)[:, 1]
        cvs.append(roc_auc_score(valid_y, valid_score))
    
    if verbose:
        print("five fold auc:", *cvs, sep=", ")
        print("mean auc:", np.mean(cvs))
        
    return cvs

In [87]:
def train_rf(X, y, param):
    clf = RandomForestClassifier(**param)
    clf.fit(X, y)
    return clf

In [91]:
param = {'n_estimators':233, 'criterion':'entropy', 'max_depth':10, 'min_samples_split':45, 'min_samples_leaf':2, \
         'max_features':0.2,'n_jobs':-1, 'random_state':42, 'class_weight':'balanced', 'max_samples':0.4}
# param.update(optuna_search.best_params_)
cvs = train_rf_cv(train.drop(['customer_phone', 'order_status_key'], axis=1), train['order_status_key'], param)

five fold auc:, 0.7678726724723317, 0.73861741235542, 0.7696763202725725, 0.7370939640564215, 0.7442838731234636
mean auc: 0.7515088484560419


In [92]:
clf = train_rf(train.drop(['order_status_key', 'customer_phone', 'fold'], axis=1), train['order_status_key'], param)
pickle.dump(clf, open(config.RF, "wb"))

## XGBoost

In [ ]:
train = pd.read_csv(config.TRAIN3)
cat_features = ['gender', 'degree', 'card_num']
train[cat_features] = train[cat_features].astype('category')

In [ ]:
def objective(trial):
    data = train.copy()
    param = {
        'booster':'gbtree', 
        'objective':'binary:logistic',
        # 'eval_metric':roc_auc_score,
        'grow_policy':'lossguide',
        'n_estimators': 88,
        'max_leaves': 28,
        'max_depth': 4,
        'eta': 0.00039413682468496394,
        'min_child_weight': 20,
        'subsample': 0.7,
        'colsample_bytree': 0.4,
        'scale_pos_weight': 6.0,
        'max_bin': 140,
        'random_state':42,
        'tree_method':'approx',
        'enable_categorical':True
        # 'early_stopping_rounds':10
    }
    
    # Add a callback for pruning.
    # pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-auc")
    aucs = []
    for fold in range(0, 5):
        train_fold = data.query(f"fold != {fold}").drop('fold', axis=1)
        valid_fold = data.query(f"fold == {fold}").drop('fold', axis=1)
        train_x, train_y = train_fold.drop('order_status_key', axis=1), train_fold['order_status_key']
        valid_x, valid_y = valid_fold.drop('order_status_key', axis=1), valid_fold['order_status_key'] 
        clf = XGBClassifier(**param).fit(train_x, train_y, verbose=False)
        preds = clf.predict_proba(valid_x)[:, 1]
        value = roc_auc_score(valid_y, preds)
        aucs.append(value)
    return np.mean(aucs)


if __name__ == "__main__":
    study = optuna.create_study(
        # pruner=optuna.pruners.MedianPruner(n_warmup_steps=10), 
        direction="maximize"
    )
    study.optimize(objective, n_trials=100)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

In [ ]:
param = {
        'booster':'gbtree', 
        'objective':'binary:logistic',
        'grow_policy':'lossguide',
        'n_estimators': 88,
        'max_leaves': 28,
        'max_depth': 4,
        'eta': 0.00039413682468496394,
        'min_child_weight': 20,
        'subsample': 0.8,
        'colsample_bytree': 0.4,
        'scale_pos_weight': 9,
        'max_bin': 140,
        'random_state':42,
        'tree_method':'approx',
        'enable_categorical':True}
train_x, train_y = train.drop(['order_status_key', 'fold'], axis=1), train['order_status_key']
clf = XGBClassifier(**param).fit(train_x, train_y)

In [ ]:
clf.save_model(config.XGBOOST)

## Catboost

In [ ]:
train = pd.read_csv(config.TRAIN3)
cat_features= ['gender', 'degree', 'card_num']
train[cat_features] = train[cat_features].astype('category')

In [ ]:
Number of finished trials: 100
value: 0.7557021810734862 and parameters: {'iterations': 220, 'learning_rate': 0.02200043117064521}

In [ ]:
def objective(trial):
    data = train.copy()
    param_distribution = {
        'loss_function':'Logloss',
        'iterations':trial.suggest_int('iterations', 200, 250, step=5),
        'learning_rate':trial.suggest_float('learning_rate', 1e-2, 0.05, log=True),
        # 'bagging_temperature':trial.suggest_float('bagging_temperature', 1e-2, 1e2, log=True),
        'subsample':0.7,
        # 'sampling_frequency':'PerTree',
        'depth':trial.suggest_int('depth', 5, 10, step=1),
        'grow_policy':'Lossguide',
        # 'min_data_in_leaf':trial.suggest_int('min_data_in_leaf', 10, 20, step=1),
        'max_leaves':trial.suggest_int('max_leaves', 15, 20, step = 1),
        'scale_pos_weight':trial.suggest_float('scale_pos_weight', 0.5, 20, log=True),
        # 'border_count':trial.suggest_int('border_count', 100, 300, step=10),
        'random_seed':42,
        'verbose':False
    }
    
    cvs = []
    clf = CatBoostClassifier(**param_distribution)
    for fold in range(0, 5):
        train_fold = data.query(f'fold !={fold}')
        valid_fold = data.query(f'fold == {fold}')
        train_x = train_fold.drop(columns=['fold', 'order_status_key'])
        train_y = train_fold['order_status_key']
        train_data = Pool(train_x, train_y, cat_features= ['gender', 'degree', 'card_num'])
        valid_x = valid_fold.drop(columns=['fold', 'order_status_key'])
        valid_y = valid_fold['order_status_key']
        valid_data = Pool(valid_x, valid_y, cat_features= ['gender', 'degree', 'card_num'])
        valid_score = clf.fit(train_data).predict_proba(valid_data)[:, 1]
        cvs.append(roc_auc_score(valid_y, valid_score))
        
    return np.mean(cvs)


study = optuna.create_study(
        # pruner=optuna.pruners.MedianPruner(n_warmup_steps=10), 
        direction="maximize"
    )
study.optimize(objective, n_trials=100)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
param = {'loss_function':'Logloss', 'iterations':225, 'learning_rate':0.020248583615876215, 'depth':7, 'max_leaves':17, 'scale_pos_weight': 1.0350162147070745, 
         'subsample':0.7, 'grow_policy':'Lossguide', 'verbose':False}
train_x, train_y = train.drop(['order_status_key', 'fold'], axis=1), train['order_status_key']
train_data = Pool(train_x, train_y, cat_features= ['gender', 'degree', 'card_num'])
clf = CatBoostClassifier(**param).fit(train_data)
clf.save_model(config.CATBOOST)

## LightGBM

In [95]:
train = pd.read_csv(config.TRAIN3).drop(['customer_phone'], axis=1)

In [ ]:
num_iterations:236
learning_rate:0.01626253894892189
num_leaves:8
max_depth:16
min_data_in_leaft:17
max_bin:420
min_data_in_bin:9
feature_fraction:0.3666772545931758

In [104]:
def objective(trial):
    data = train.copy()
    
    param_distribution = {
        'objective': 'binary',
        'num_iterations': trial.suggest_int('num_iterations', 200, 300, log=True),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.05, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 5, 10, step=1),
        'max_depth': trial.suggest_int('max_depth', 15, 20, step=1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaft', 15, 20, step=1),
        'max_bin': trial.suggest_int('max_bin', 420, 450, step=5),
        'min_data_in_bin': trial.suggest_int('min_data_in_bin', 5, 10, step=1),
        # 'bagging_fraction':trial.suggest_float('bagging_fraction', 0.3, 1, log=True),
        'neg_bagging_fraction':trial.suggest_float('neg_bagging_fraction', 0.3, 1, log=True),
        # 'cat_smooth': trial.suggest_float('cat_smooth', 1, 1e3, log=True),
        'feature_fraction':trial.suggest_float('feature_fraction', 0.3, 1, log=True),
        'verbosity':0,
        'force_col_wise':True,
        'linear_tree':True,
        'seed':42
    }
    
    aucs = []
    for fold in range(0, 5):
        train_fold = data.query(f'fold != {fold}')
        train_x, train_y = train_fold.drop(columns=['fold', 'order_status_key'], axis=1), train_fold['order_status_key']
        dtrain = lgbm.Dataset(train_x, train_y)
        valid_fold = data.query(f'fold == {fold}')
        valid_x, valid_y = valid_fold.drop(columns=['fold', 'order_status_key'], axis=1), valid_fold['order_status_key']
        dvalid = lgbm.Dataset(valid_x, valid_y)
        clf = lgbm.train(param_distribution, dtrain, categorical_feature=['gender', 'card_num', 'degree', 'als_m12_id_nbank_orgnum'])
        valid_score = clf.predict(valid_x)
        aucs.append(roc_auc_score(valid_y, valid_score))
    
    return np.mean(aucs)


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

print("Best trial")
trial = study.best_trial

print(f"Avg of five fold auc:{trial.value:.6f}")

print("Params")
for key, value in trial.params.items():
    print(f"{key}:{value}")

[I 2023-05-25 18:15:56,572] A new study created in memory with name: no-name-03699222-54b6-4b2c-b2de-a1007a5835d2
[I 2023-05-25 18:15:57,645] Trial 0 finished with value: 0.7542673462652996 and parameters: {'num_iterations': 268, 'learning_rate': 0.03763788859826995, 'num_leaves': 6, 'max_depth': 15, 'min_data_in_leaft': 19, 'max_bin': 430, 'min_data_in_bin': 7, 'neg_bagging_fraction': 0.9852508880898316, 'feature_fraction': 0.7322002090709276}. Best is trial 0 with value: 0.7542673462652996.
[I 2023-05-25 18:15:59,070] Trial 1 finished with value: 0.7496137163385694 and parameters: {'num_iterations': 212, 'learning_rate': 0.008920880273165116, 'num_leaves': 5, 'max_depth': 16, 'min_data_in_leaft': 18, 'max_bin': 430, 'min_data_in_bin': 8, 'neg_bagging_fraction': 0.7931179786088892, 'feature_fraction': 0.6698077019418888}. Best is trial 0 with value: 0.7542673462652996.
[I 2023-05-25 18:16:00,184] Trial 2 finished with value: 0.7545335085034868 and parameters: {'num_iterations': 200, '

Best trial
Avg of five fold auc:0.755611
Params
num_iterations:231
learning_rate:0.020354684816987482
num_leaves:8
max_depth:19
min_data_in_leaft:16
max_bin:445
min_data_in_bin:9
neg_bagging_fraction:0.4810778112820544
feature_fraction:0.8231745626945346


In [101]:
param = {'num_iterations':300, 'learning_rate':0.018672121641626984, 'num_leaves':7,' max_depth':18, 'min_data_in_leaf':17
        ,'max_bin':445, 'min_data_in_bin':8, 'objective':'binary', 'neg_bagging_fraction':0.5843897307492004,
         'feature_fraction':0.4826469048366353, 'verbosity':0, 'force_col_wise':True, 
          'linear_tree':False, 'seed':42}
train_x, train_y = train.drop(['order_status_key', 'fold'], axis=1), train['order_status_key']
dtrain = lgbm.Dataset(train_x, label=train_y)
clf = lgbm.train(param, dtrain, categorical_feature=['degree', 'card_num', 'gender'])
clf.save_model(config.LIGHTGBM)

## learning curve

In [ ]:
train = pd.read_csv(config.TRAIN)
train_x, train_y = train.drop(['order_status_key', 'fold'], axis=1), train['order_status_key']
lr_param = {'C':1, 'max_iter':20, 'solver':'newton-cg', 'class_weight':'balanced'}
rf_param = {'n_estimators':358, 'criterion':'entropy', 'max_depth':9, 'min_samples_split':35, 'min_samples_leaf':3, \
         'max_features':0.2,'n_jobs':-1, 'random_state':42, 'class_weight':'balanced', 'max_samples':0.4}
lr = LogisticRegression(**lr_param)
rf= RandomForestClassifier(**rf_param)
common_params = {
    "X": train_x,
    "y": train_y,
    "train_sizes": np.linspace(0.1, 1.0, num=10),
    "cv": StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    "score_type": "both",
    "n_jobs": 4,
    "line_kw": {"marker": "o"},
    "std_display_style": "fill_between",
    "score_name": "roc_auc",
}


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 6))
for ax_idx, estimator in enumerate([lr, rf]):
    LearningCurveDisplay.from_estimator(estimator, **common_params, ax=ax[ax_idx])
    ax[ax_idx].set_title(f"learning curve for {estimator.__class__.__name__}")

通过分析学习曲线可知，增加训练样本不能改善样本性能，下一步的精力应该放在改善模型/特征工程/收集更多特征上